In [1]:
#load packages I need
import os
import tools
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import pyscenic
from pyscenic.aucell import aucell

In [2]:
#show current directory 
os.getcwd()
os.chdir('/scratch/user/s4436039/scdata/Python_Integration_Sep')
os.getcwd()

'/scratch/user/s4436039/scdata/Python_Integration_Sep'

In [3]:
# read in data
data = sc.read_h5ad('NRclean_clustered2.h5ad')

In [4]:
#make two versions of the data, once containing just cDC1 and mregDC, and another containing just cDC2 and mregDC
data_DC1mreg = data[data.obs["NR_annotations_simple"].isin(["cDC1","mregDC"])]
data_DC2mreg = data[data.obs["NR_annotations_simple"].isin(["cDC2","mregDC"])]

# pyscenic:

## 1. GRN Inference (gene regulatory network)

In [26]:
from arboreto.algo import grnboost2

In [23]:
# Extract the gene expression matrix
expr_matrix = pd.DataFrame(data_DC1mreg.X, index=data_DC1mreg.obs_names, columns=data_DC1mreg.var_names)

In [24]:
expr_matrix.head()

,HES4,ISG15,TNFRSF18,TNFRSF4,ATAD3C,PRKCZ,HES2,TNFRSF25,ERRFI1,RBP7,...,MX1,TFF3,TFF1,CSTB,COL6A1,COL6A2,S100B,MT-ATP8,MT-ND4L,MT-ND6
GSE215120_AM1_ACCAGTAAGACTGGGT-1,-0.323357,0.228329,-0.125734,-0.165770,-0.070054,-0.084578,-0.075738,-0.079652,2.267030,-0.346255,...,-0.764149,-0.100872,-0.082936,-1.006041,-0.121596,-0.109659,-0.321856,-0.552289,-1.038093,0.623567
GSE215120_AM1_ACGTCAACAAGGACTG-1,-0.392818,0.613830,-0.106020,-0.234417,-0.095923,-0.092989,-0.196529,-0.061677,-0.348859,-0.290299,...,-0.107679,-0.014869,-0.042852,-0.659405,-0.249138,-0.201969,1.861450,-0.423993,-1.248492,-0.433341
GSE215120_AM1_ACTGAGTCAGGCTGAA-1,-0.306740,-0.984695,-0.125048,5.608533,-0.069898,-0.083811,-0.078175,-0.084302,-0.226265,-0.330249,...,-0.754118,-0.099942,-0.081539,-0.916316,-0.121152,-0.109597,-0.321951,-0.565476,-1.084679,-0.540388
GSE215120_AM1_ACTGTCCGTCTCTTAT-1,-0.320882,-0.364538,-0.125863,-0.174040,-0.070967,-0.083985,-0.083788,-0.082307,-0.248738,-0.361046,...,-0.773888,-0.085153,-0.068670,-1.037655,-0.119181,-0.108784,1.384639,-0.555693,-1.035306,-0.471768
GSE215120_AM1_ATAACGCGTAGCGCTC-1,-0.335533,-0.546188,-0.122833,-0.178790,-0.074465,-0.085767,-0.097129,-0.076660,-0.264832,-0.342982,...,-0.832834,-0.082075,-0.072016,0.219361,-0.140774,-0.123798,1.022691,-0.580199,-1.059602,-0.538614


In [27]:
# Run GRN inference
# adjacency list --> potential regulatory relationships between transcription factors and their target genes, identified based on gene expression patterns.
adjacencies = grnboost2(expression_data=expr_matrix, gene_names=data_DC1mreg.var_names)

/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/distributed/client.py:1613: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| pandas  | 2.2.2  | 2.2.2     | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


TypeError: Must supply at least one delayed object

In [ ]:
# Save results
adjacencies.to_csv("adjacencies.tsv", sep="\t", index=False)

# following tute:

In [5]:
os.chdir('/scratch/user/s4436039/scdata/PySCENIC')
os.getcwd()

'/scratch/user/s4436039/scdata/PySCENIC'

In [6]:
# transcription factors list
f_tfs = "allTFs_hg38.txt" 

In [9]:
#make expression matrix
expr_matrix = pd.DataFrame(data_DC1mreg.X, index=data_DC1mreg.obs_names, columns=data_DC1mreg.var_names)

In [12]:
expr_matrix.to_csv('DC1mreg_expr_matrix.tsv', sep='\t', index=False)

In [13]:
f_expr = "DC1mreg_expr_matrix.tsv" 

In [15]:
import loompy as lp
import numpy as np

In [17]:
# Save the path where you want loom file to save (current directory)
f_loom_path_DC1mreg = "DC1mreg_loom.loom"

In [18]:
row_attrs = { 
    "Gene": np.array(data_DC1mreg.var.index) ,
}
col_attrs = { 
    "CellID":  np.array(data_DC1mreg.obs.index) ,
    "nGene": np.array( np.sum(data_DC1mreg.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(data_DC1mreg.X.transpose() , axis=0)).flatten() ,
}

lp.create( f_loom_path_DC1mreg, data_DC1mreg.X.transpose(), row_attrs, col_attrs )

In [19]:
!pyscenic grn {f_loom_path_DC1mreg} {f_tfs} -o adj.csv --num_workers 20

Traceback (most recent call last):
  File "/home/s4436039/miniforge3/envs/env/bin/pyscenic", line 5, in <module>
    from pyscenic.cli.pyscenic import main
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pyscenic/cli/pyscenic.py", line 19, in <module>
    from arboreto.algo import genie3, grnboost2
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/arboreto/algo.py", line 5, in <module>
    import pandas as pd
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/__init__.py", line 49, in <module>
    from pandas.core.api import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/core/api.py", line 47, in <module>
    from pandas.core.groupby import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/core/groupby/__init__.py", line 1, in <module>
    from pandas.core.groupby.generic import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packag

In [14]:
!pyscenic grn {f_expr} {f_tfs} -o adj.csv --num_workers 20

Traceback (most recent call last):
  File "/home/s4436039/miniforge3/envs/env/bin/pyscenic", line 5, in <module>
    from pyscenic.cli.pyscenic import main
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pyscenic/cli/pyscenic.py", line 19, in <module>
    from arboreto.algo import genie3, grnboost2
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/arboreto/algo.py", line 5, in <module>
    import pandas as pd
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/__init__.py", line 49, in <module>
    from pandas.core.api import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/core/api.py", line 47, in <module>
    from pandas.core.groupby import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packages/pandas/core/groupby/__init__.py", line 1, in <module>
    from pandas.core.groupby.generic import (
  File "/home/s4436039/miniforge3/envs/env/lib/python3.12/site-packag

## 2. Motif enrichment and pruning the GRN (based on motif enrichment)
* to perform pruning need motif rankings and TF lists

In [ ]:
# Replace cisTarget_db/*.feather with the path to your downloaded motif rankings files.
# Replace expression.csv with the gene expression matrix.

In [ ]:
pyscenic ctx adjacencies.tsv cisTarget_db/*.feather --annotations_fname motifs-v9-nr.hgnc-m0.001-o0.0.tbl --expression_mtx_fname expression.csv --output pruned_networks.csv --num_workers 4

## 3. Scoring the cells (using AUCell to determine activity)

# other

# using pyscenic to extract the regulons
* downloaded necessary databases: https://pyscenic.readthedocs.io/en/latest/installation.html
* database ranking the genome --> used Homo sapiens - hg38 - refseq_r80 - SCENIC+ databases - Gene based --> hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
* Motif to TF annotations --> 

In [ ]:
# using pyscenic to extract the regulons